In [1]:
# Note using suppress to reduce constant red warning boxes
#TODO FABIO check if packages must be installed for reproducibility
suppressPackageStartupMessages({
    suppressWarnings({
        library(tidyverse)
        library(repr)
        library(tidymodels)
        library(tidyr)
        library(dplyr)
    })
})


In [2]:
# Size and general style set up
options(repr.plot.width = 6, repr.plot.height = 4, repr.matrix.max.rows = 7,readr.show_col_types = FALSE)

# Load Data
player_data <- read_csv("https://raw.githubusercontent.com/FabianoGLentini/player-subscription-ml/refs/heads/main/data/players.csv")
# TODO may not need session data
# session_data <- read_csv("")



# Data Science Project: Project Final Report


## Introduction:

## Methods & Results:

### Set up and intro to data: "TODO should rename later." 

In [3]:
# Wrangle data
player_df <- player_data |>
            mutate(
                subscribe = as_factor(subscribe),  # lgl -> fct for analysis and modeling
                gender = as_factor(gender),        # chr -> fct for analysis and modeling
                experience = as_factor(experience) # chr -> fct for analysis and modeling
            ) |>
            select( subscribe, gender, played_hours, experience, Age) |>
            drop_na() # Removed row with NA values, as it may distort the model      

# TODO FABIO check if player_hours should adjust to use a binary outcome, 
# either played or didn't play at all, or if any of the predictor should be removed.


### Training and Analysis:

In [4]:
head(player_df) # TODO DELETE tmp  for set up 

subscribe,gender,played_hours,experience,Age
<fct>,<fct>,<dbl>,<fct>,<dbl>
TRUE,Male,30.3,Pro,9
TRUE,Male,3.8,Veteran,17
FALSE,Male,0.0,Veteran,17
TRUE,Female,0.7,Amateur,21
TRUE,Male,0.1,Regular,21
TRUE,Female,0.0,Amateur,17


In [5]:
set.seed(2025) # Don't change
# Prep for modelling
ply_df <- select(player_df, Age, subscribe, gender, played_hours)  # Exclude row_id & experience for modeling purposes

# Split step
player_split <- initial_split(ply_df, prop = 0.75, strata = subscribe) 
player_train <- training(player_split)
player_test <- testing(player_split)
player_train

Age,subscribe,gender,played_hours
<dbl>,<fct>,<fct>,<dbl>
17,FALSE,Male,0.0
21,FALSE,Male,0.0
22,FALSE,Male,0.0
17,FALSE,Female,0.1
⋮,⋮,⋮,⋮
17,TRUE,Female,0
20,TRUE,Male,0
17,TRUE,Female,0


In [6]:
#TODO FABIO write up split/scale/recipe step reasoning
# 'initial split' Use Strata sub to balance the outcome for the bool prediction to avoid 
# imbalance in our test and train data split

# ... why use 75 25 split instead of 70/30 etc?

# ~ maybe explain why start with all predictors

In [7]:
# TODO FABIO confirm that not having set.seed here is fine as long as set.seed above code cell goes first
# K-fold cross-validation
kfolds <- vfold_cv(player_train, v = 10, strata = subscribe)


In [8]:
#TODO FABIO write reasonin + graph to show fold outcome
#Note the reason of using 10 10-fold is due to the small size data,
#hence it will improve the estimate and 

In [9]:
#TODO FABIO ... search refractor options to reduce code clutter

# Recipes:
# Scale/Recipe

# Note: A = Age, G = gender and H = played_hours
# Recipe 01:
# Age + gender + played_hours
player_recipe_All <- recipe(subscribe ~ Age + gender + played_hours, data = player_train) |>
            step_dummy(all_nominal_predictors()) |>
            step_zv(all_predictors) |> # Used to remove zero-variance variable after wrangling, gender_Other category had no rows associated to i 
            step_normalize(all_numeric_predictors())
player_recipe_All
# # Recipe 02:
# # Age + played_hours
# player_recipe_AH <- recipe(subscribe ~ Age + played_hours, player_train) |>
#                 step_normalize(all_predictors())

# # Recipe 03:
# # Age + gender 
# player_recipe_AG <- recipe(subscribe ~ Age + gender, player_train) |>
#                 step_normalize(all_predictors())

# # Recipe 04:
# # gender + played_hours
# player_recipe_GH <- recipe(subscribe ~ gender + played_hours, player_train) |>
#                 step_normalize(all_predictors())

# # Recipe 05:
# # Age
# player_recipe_A <- recipe(subscribe ~ Age, player_train) |>
#                 step_normalize(all_predictors())

# # Recipe 06:
# # gender
# player_recipe_G <- recipe(subscribe ~ gender, player_train) |>
#                 step_normalize(all_predictors())

# # Recipe 07:
# # played_hours
# player_recipe_H <- recipe(subscribe ~ played_hours, player_train) |>
#                 step_normalize(all_predictors())





── Recipe ──────────────────────────────────────────────────────────────────────



── Inputs 

Number of variables by role

outcome:   1
predictor: 3



── Operations 

• Dummy variables from: all_nominal_predictors()

• Zero variance filter on: all_predictors

• Centering and scaling for: all_numeric_predictors()



In [10]:
#TODO FABIO breackdown hypothesis for each recipe variation ...?

In [11]:
# Spec set up
player_spec <- nearest_neighbor(weight_func = "rectangular", neighbors =3) |>
        set_engine("kknn") |>
        set_mode("classification")

#TODO FABIO ... search refractor options to reduce code clutter
# Check mean and standard error through collect_metrics


In [12]:
# TODO FABIO ~ write up spec use and impl of vfold..?

In [13]:
#TODO FABIO ... search refractor options to reduce code clutter/redundancy 
# Workflow Step

# TODO FABIO look into increasing it and the impact on results
# k_grid
k_grid = tibble(neighbors = seq(from = 1, to = 10, by = 1))

# Note: A = Age, G = gender and H = played_hours
# Workflow 01:
# Age + gender + played_hours
player_wf_All <- workflow()  |>
            add_recipe(player_recipe_All) |>
            add_model(player_spec) |>
            fit(player_train)

player_wf_All
# player_wf_All
# # Workflow 02:
# # Age + played_hours
# player_wf_AH <- workflow()  |>
#             add_recipe(player_recipe_AH) |>
#             add_model(player_spec)

# # Workflow 03:
# # Age + gender 
# player_wf_AG <- workflow()  |>
#             add_recipe(player_recipe_AG) |>
#             add_model(player_spec)

# # Workflow 04:
# # gender + played_hours
# player_wf_GH <- workflow()  |>
#             add_recipe(player_recipe_GH) |>
#             add_model(player_spec)


# # Workflow 05:
# # Age
# player_wf_A <- workflow()  |>
#             add_recipe(player_recipe_A) |>
#             add_model(player_spec)


# # Workflow 06:
# # gender
# player_wf_G <- workflow()  |>
#             add_recipe(player_recipe_G) |>
#             add_model(player_spec)

# # Workflow 07:
# # played_hours
# player_wf_H <- workflow()  |>
#             add_recipe(player_recipe_H) |>
#             add_model(player_spec)


ERROR: [1m[33mError[39m in `step_zv()`:[22m
[1mCaused by error in `prep()`:[22m
[33m![39m Can't select columns that don't exist.
[31m✖[39m Column `all_predictors` doesn't exist.


In [ ]:
#TODO FABIO write workflow step use case/what it functionally is doing 

### Player Type Exploration: "TODO may need renaming"

## Discussion:

In [ ]:
#TODO tmp info grounds
?tune_grid